# Multi Class Logistic Regression

# OVR - One Over Rest

In [5]:
from sklearn import datasets
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
# loading data
iris = datasets.load_iris()

# sperate training and test
msk = np.random.rand(len(iris.target)) < 0.8
train_X = iris.data[msk]
train_Y = iris.target[msk]

test_X = iris.data[~msk]
test_Y = iris.target[~msk]

# regularization
reg = 0.01

# multi-class stretegies
# One Vs All(Rest) (OVA) -- default
OVRModel = LogisticRegression(C=reg, multi_class='ovr', solver='newton-cg',class_weight=None)
OVRModel.fit(train_X, train_Y)

# predict test
predVals = OVRModel.predict(test_X)

# checking acc
totalNum = len(test_Y)
truePredCount = sum(np.equal(predVals, test_Y))
print('Report for One Vs Rest Strategy')
print(classification_report(predVals, test_Y))
print("Accuracy :: ",accuracy_score(predVals, test_Y))
labels = [0,1,2]
pd.DataFrame(confusion_matrix(predVals, test_Y),index=labels, columns=labels)


Report for One Vs Rest Strategy
             precision    recall  f1-score   support

          0       1.00      0.71      0.83         7
          1       0.40      1.00      0.57         4
          2       1.00      0.78      0.88        18

avg / total       0.92      0.79      0.82        29

Accuracy ::  0.793103448276


,0,1,2
0,5,2,0
1,0,4,0
2,0,4,14


# One Vs One
# One Vs OnO is a voting strategy based on comparing each model to another model


In [6]:
# Creating voter models which compares one class to another one
setOfClasses = set(iris.target)
numberOfClasses = len(setOfClasses)
classList = list(setOfClasses)
modelList = []
for i in classList:
    for j in classList:
        if i < j:
            flt = np.logical_or(np.equal(train_Y, i), np.equal(train_Y, j))
            subX = train_X[flt]
            subY = train_Y[flt]
            subModel = LogisticRegression(C=reg)
            subModel.fit(subX, subY)
            modelList.append(subModel)

# Counting of votes of models for determining classes of test set

predVals = []
for testIns in test_X:
    votes = np.zeros(numberOfClasses)
    # print votes

    for currModel in modelList:
        predVal = currModel.predict([testIns])
        tmpIn = classList.index(predVal)
        votes[tmpIn] = votes[tmpIn] + 1

    predVals.append(classList[votes.argmax()])


# print results
totalNum = len(test_Y)
truePredCount = 0
for pred, trueClass in zip(predVals,test_Y):
    if pred == trueClass:
        truePredCount = truePredCount + 1

print('Report for One Vs One Strategy')
print(classification_report(predVals, test_Y))
print("Accuracy :: ",accuracy_score(predVals, test_Y))

labels = [0,1,2]
pd.DataFrame(confusion_matrix(predVals, test_Y),index=labels, columns=labels)


Report for One Vs One Strategy
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       0.80      1.00      0.89         8
          2       1.00      0.88      0.93        16

avg / total       0.94      0.93      0.93        29

Accuracy ::  0.931034482759


,0,1,2
0,5,0,0
1,0,8,0
2,0,2,14
